## Deliverable 2. Create a Customer Travel Destinations Map.

In [11]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Altoona,US,40.5187,-78.3947,88.81,59,39,1.01,scattered clouds
1,1,Barrow,US,71.2906,-156.7887,44.62,87,90,10.36,overcast clouds
2,2,Upernavik,GL,72.7868,-56.1549,41.25,81,12,13.24,few clouds
3,3,Kodiak,US,57.7900,-152.4072,51.31,87,90,6.91,fog
4,4,Valparaiso,CL,-33.0393,-71.6273,58.64,82,90,6.91,drizzle


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             232
City                   232
Country                231
Lat                    232
Lng                    232
Max Temp               232
Humidity               232
Cloudiness             232
Wind Speed             232
Current Description    232
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
updated_cities_df = preferred_cities_df.dropna()
updated_cities_df.count()

Unnamed: 0             231
City                   231
Country                231
Lat                    231
Lng                    231
Max Temp               231
Humidity               231
Cloudiness             231
Wind Speed             231
Current Description    231
dtype: int64

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = updated_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Altoona,US,88.81,scattered clouds,40.5187,-78.3947,
6,Atuona,PF,77.97,broken clouds,-9.8000,-139.0333,
11,Tiruvannamalai,IN,83.23,scattered clouds,12.2167,79.0667,
15,Bosaso,SO,89.04,broken clouds,11.2842,49.1816,
24,Vanimo,PG,75.42,overcast clouds,-2.6741,141.3028,
25,Georgetown,MY,83.89,few clouds,5.4112,100.3354,
31,Pacific Grove,US,83.80,few clouds,36.6177,-121.9166,
32,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,
33,Kavieng,PG,81.00,light rain,-2.5744,150.7967,
35,Seoul,KR,81.97,clear sky,37.5683,126.9778,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat= row["Lat"]
    lng= row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace = True)
hotel_df.dropna(subset = ["Hotel Name"], inplace = True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Altoona,US,88.81,scattered clouds,40.5187,-78.3947,Wingate by Wyndham Altoona Downtown/Medical Ce...
6,Atuona,PF,77.97,broken clouds,-9.8000,-139.0333,Villa Enata
11,Tiruvannamalai,IN,83.23,scattered clouds,12.2167,79.0667,Hotel Aakash
15,Bosaso,SO,89.04,broken clouds,11.2842,49.1816,Red Sea Hotel
24,Vanimo,PG,75.42,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
...,...,...,...,...,...,...,...
652,Zhanaozen,KZ,85.60,clear sky,43.3000,52.8000,K.korabay 28A
654,Tecoanapa,MX,87.22,broken clouds,16.5167,-98.7500,Hotel Manglares
656,Eyl,SO,77.20,clear sky,7.9803,49.8164,Amiin Hotel
657,Maunabo,PR,85.69,overcast clouds,18.0072,-65.8993,Maunacaribe


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacations.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))